![img](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/CRISP-DM_Process_Diagram.png/598px-CRISP-DM_Process_Diagram.png)

# Objective?

Get all the descriptions of the Products and their department

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
import random

## Links

We are gonna gather all the links from the sections that we want and when we find 'Agencia de viajes' We brake the loop because we dont want the links down it

In [2]:
url = 'https://www.elcorteingles.es/'

req = requests.get(url)

soupHome = BeautifulSoup(req.text)

i = 0
links = []

for a in soupHome.find_all('a', href=True):
    if ('viajeselcorteingles' in a['href']):
        break
    if all([term not in a['href'] for term in ['/supermercado/', '/entradas/', '/club-del-gourmet/']]):
        links.append(a['href'].split('?')[0])

#Remove duplicates
links = list(dict.fromkeys(links))

In [3]:
len(links)

1259

In [12]:
links[980]

'/jane/bebes/'

## Correcciones manuales

Estos links corresponden a listas de prodcutos que nos producen duplicados, si las filtramos de inicio optimizamos el proceso al tener que hacer menos peticiones.

In [5]:
links_to_remove = ['/75/','/moda/', '/moda/mujer/', '/moda/zapatos/', '/cine/', '/musica/', '/mascotas/', '/bricor/',
                  '/bricor/iluminacion/', '/bricor/herramientas/', '/bricor/estanterias-y-ordenacion/', '/juguetes/'
                  '/bricor/armarios/', '/electronica/', '/electrodomesticos/', '/deportes/', '/hogar/', '/libros/',
                  '/bricor/bano/','/bricor/cocinas/',' /bricor/armarios/', 'https://www.elcorteingles.es/perfumeria/']
for x in links_to_remove:
    try:
        links.remove(x)
    except Exception as e:
        print(f'Exception: {e}')
        

Exception: list.remove(x): x not in list
Exception: list.remove(x): x not in list
Exception: list.remove(x): x not in list


## Productos

In [19]:
products = []
url_errors = []
suffix = ''
    
for url in links:
    url = 'https://www.elcorteingles.es'+url #Juguetes juegos de mesa y habilidad
    print (url)
    for page in range (1,15):
        time.sleep(random.randint(0,1)+random.random()) #Para evitar suponerle un daño a la empresa
        try:
            if page > 1:
                suffix = '/'+str(page)+'/'
                
            req = requests.get(url+suffix, timeout=10)

            if req.ok: # status_code == 200
                s = BeautifulSoup(req.text)
                prods = s.findAll('span')

                for n, span_ in enumerate(prods):
                    if 'data-json' in span_.attrs and ('data-scope' in span_.attrs and span_.attrs['data-scope'] == 'product'):
                        obj = json.loads(span_['data-json'])
                        obj['image'] = 'http:{}'.format(str(span_.parent.find('img')['src']))
                        products.append(obj)

            else: # imprime url y status code donde ha dado error
                print(f'Status code error on url {url}. Status code: {req.status_code}')
                url_errors.append(url)
                break # Si no hay mas pages no sigue buscando mas y pasa al siguiente link
                
        except Exception as e:
            print(f'Exception: {e}')
            if url not in url_errors:
                url_errors.append(url)
                
    time.sleep(random.randint(3,6) + random.random()) #Para evitar suponerle un daño a la empresa
                


https://www.elcorteingles.es/moda/mujer/abrigos/
https://www.elcorteingles.es/moda/mujer/camisas-blusas-y-tops/
https://www.elcorteingles.es/moda/mujer/camisetas/
https://www.elcorteingles.es/moda/mujer/cazadoras/
Status code error on url https://www.elcorteingles.es/moda/mujer/cazadoras/. Status code: 400
https://www.elcorteingles.es/moda/mujer/chalecos/
Status code error on url https://www.elcorteingles.es/moda/mujer/chalecos/. Status code: 400
https://www.elcorteingles.es/moda/mujer/chaquetas-y-blazers/
Status code error on url https://www.elcorteingles.es/moda/mujer/chaquetas-y-blazers/. Status code: 400
https://www.elcorteingles.es/moda/mujer/faldas/
Status code error on url https://www.elcorteingles.es/moda/mujer/faldas/. Status code: 400
https://www.elcorteingles.es/moda/mujer/monos/
Status code error on url https://www.elcorteingles.es/moda/mujer/monos/. Status code: 400
https://www.elcorteingles.es/moda/mujer/pantalones/
https://www.elcorteingles.es/moda/mujer/polos/
Status co

Exception: HTTPSConnectionPool(host='www.elcorteingles.eshttps', port=443): Max retries exceeded with url: //www.elcorteingles.es/couchel/moda/mujer-tallas-grandes//24/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000229B9293B38>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
Exception: HTTPSConnectionPool(host='www.elcorteingles.eshttps', port=443): Max retries exceeded with url: //www.elcorteingles.es/couchel/moda/mujer-tallas-grandes//25/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000229B9293D68>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
Exception: HTTPSConnectionPool(host='www.elcorteingles.eshttps', port=443): Max retries exceeded with url: //www.elcorteingles.es/couchel/moda/mujer-tallas-grandes//26/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000229B9293860>: Failed to establish a new

Status code error on url https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/gorros-y-sombreros/. Status code: 400
https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/mochilas/
Status code error on url https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/mochilas/. Status code: 400
https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/monederos/
Status code error on url https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/monederos/. Status code: 400
https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/neceseres/
Status code error on url https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/neceseres/. Status code: 400
https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/pajaritas/
Status code error on url https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/pajaritas/. Status code: 400
https://www.elcorteingles.es/moda/hombre/accesorios-de-hombre/panuelos/
Status code error on url https://www.elcorteingle

https://www.elcorteingles.es/moda/zapatillas-deportivas/
Status code error on url https://www.elcorteingles.es/moda/zapatillas-deportivas/. Status code: 400
https://www.elcorteingles.es/moda/espacio-designers/zapatos/
https://www.elcorteingles.es/exclusivos-online-zapatos/moda/zapatos/
Status code error on url https://www.elcorteingles.es/exclusivos-online-zapatos/moda/zapatos/. Status code: 404
https://www.elcorteingles.es/moda/zapatos/tallas-especiales-de-calzado/
https://www.elcorteingles.es/moda/tendencias/zapatos/
https://www.elcorteingles.es/moda/zapatos/marcas/
https://www.elcorteingles.es/callaghan/moda/zapatos/
https://www.elcorteingles.es/camper/moda/zapatos/
https://www.elcorteingles.es/clarks/moda/zapatos/
https://www.elcorteingles.es/dustin/moda/zapatos/
https://www.elcorteingles.es/formula-joven/moda/zapatos/
https://www.elcorteingles.es/gloria-ortiz/moda/zapatos/
https://www.elcorteingles.es/hispanitas/moda/zapatos/
https://www.elcorteingles.es/michael-michael-kors/moda/

Status code error on url https://www.elcorteingles.es/moda/lenceria/batas/. Status code: 400
https://www.elcorteingles.es/moda/lenceria/bodys/
Status code error on url https://www.elcorteingles.es/moda/lenceria/bodys/. Status code: 400
https://www.elcorteingles.es/moda/lenceria/bodys-reductores/
Status code error on url https://www.elcorteingles.es/moda/lenceria/bodys-reductores/. Status code: 400
https://www.elcorteingles.es/moda/lenceria/bragas/
https://www.elcorteingles.es/moda/lenceria/bragas-reductoras/
Status code error on url https://www.elcorteingles.es/moda/lenceria/bragas-reductoras/. Status code: 400
https://www.elcorteingles.es/moda/lenceria/calcetines/
Status code error on url https://www.elcorteingles.es/moda/lenceria/calcetines/. Status code: 400
https://www.elcorteingles.es/moda/lenceria/camisetas-reductoras/
Status code error on url https://www.elcorteingles.es/moda/lenceria/camisetas-reductoras/. Status code: 400
https://www.elcorteingles.es/moda/lenceria/camisetas-in

...

https://www.elcorteingles.es/izipizi/regalos-originales/
Status code error on url https://www.elcorteingles.es/izipizi/regalos-originales/. Status code: 403
https://www.elcorteingles.es/regalos-originales/merchandising/
Status code error on url https://www.elcorteingles.es/regalos-originales/merchandising/. Status code: 403
https://www.elcorteingles.es/regalos-originales/futbol/
Status code error on url https://www.elcorteingles.es/regalos-originales/futbol/. Status code: 403
https://www.elcorteingles.es/funko/regalos-originales/
Status code error on url https://www.elcorteingles.es/funko/regalos-originales/. Status code: 403
https://www.elcorteingles.es/tarjetaregalo/tarjetaregalo/tarjetaregalo.asp
Status code error on url https://www.elcorteingles.es/tarjetaregalo/tarjetaregalo/tarjetaregalo.asp. Status code: 403
https://www.elcorteingles.es/flores-y-regalos/
Status code error on url https://www.elcorteingles.es/flores-y-regalos/. Status code: 403
https://www.elcorteingles.es/listasd

Status code error on url https://www.elcorteingles.es/radar/cine/. Status code: 403
https://www.elcorteingles.es/cine/series-de-tv/
Status code error on url https://www.elcorteingles.es/cine/series-de-tv/. Status code: 403
https://www.elcorteingles.es/radar/seriestv/
Status code error on url https://www.elcorteingles.es/radar/seriestv/. Status code: 403
https://www.elcorteingles.es/musica/ambiental-y-de-relajacion/
Status code error on url https://www.elcorteingles.es/musica/ambiental-y-de-relajacion/. Status code: 403
https://www.elcorteingles.es/musica/bandas-sonoras/
Status code error on url https://www.elcorteingles.es/musica/bandas-sonoras/. Status code: 403
https://www.elcorteingles.es/musica/clasico/
Status code error on url https://www.elcorteingles.es/musica/clasico/. Status code: 403
https://www.elcorteingles.es/musica/country/
Status code error on url https://www.elcorteingles.es/musica/country/. Status code: 403
https://www.elcorteingles.es/musica/dance-y-electronica/
Statu

https://www.elcorteingles.es/mascotas/accesorios-para-perros/
Status code error on url https://www.elcorteingles.es/mascotas/accesorios-para-perros/. Status code: 403
https://www.elcorteingles.es/mascotas/higiene-para-perros/
Status code error on url https://www.elcorteingles.es/mascotas/higiene-para-perros/. Status code: 403
https://www.elcorteingles.es/mascotas/salud-para-perros/
Status code error on url https://www.elcorteingles.es/mascotas/salud-para-perros/. Status code: 403
https://www.elcorteingles.es/mascotas/comida-para-gatos/
Status code error on url https://www.elcorteingles.es/mascotas/comida-para-gatos/. Status code: 403
https://www.elcorteingles.es/mascotas/accesorios-para-gatos/
Status code error on url https://www.elcorteingles.es/mascotas/accesorios-para-gatos/. Status code: 403
https://www.elcorteingles.es/mascotas/higiene-para-gatos/
Status code error on url https://www.elcorteingles.es/mascotas/higiene-para-gatos/. Status code: 403
https://www.elcorteingles.es/masco

Status code error on url https://www.elcorteingles.es/nutro/mascotas/. Status code: 403
https://www.elcorteingles.es/retorn/mascotas/
Status code error on url https://www.elcorteingles.es/retorn/mascotas/. Status code: 403
https://www.elcorteingles.es/whiskas/mascotas/
Status code error on url https://www.elcorteingles.es/whiskas/mascotas/. Status code: 403
https://www.elcorteingles.es/gourmet-gold/mascotas/
Status code error on url https://www.elcorteingles.es/gourmet-gold/mascotas/. Status code: 403
https://www.elcorteingles.es/kong/mascotas/
Status code error on url https://www.elcorteingles.es/kong/mascotas/. Status code: 403
https://www.elcorteingles.es/bricor/muebles-de-jardin/
Status code error on url https://www.elcorteingles.es/bricor/muebles-de-jardin/. Status code: 403
https://www.elcorteingles.es/bricor/muebles-de-jardin/barbacoas/
Status code error on url https://www.elcorteingles.es/bricor/muebles-de-jardin/barbacoas/. Status code: 403
https://www.elcorteingles.es/bricor/

Status code error on url https://www.elcorteingles.es/bricor/estanterias-y-ordenacion/zapateros/. Status code: 403
https://www.elcorteingles.es/bricor/armarios/
Status code error on url https://www.elcorteingles.es/bricor/armarios/. Status code: 403
https://www.elcorteingles.es/bricor/armarios/armarios-de-resina/
Status code error on url https://www.elcorteingles.es/bricor/armarios/armarios-de-resina/. Status code: 403
https://www.elcorteingles.es/bricor/armarios/armarios-despenseros/
Status code error on url https://www.elcorteingles.es/bricor/armarios/armarios-despenseros/. Status code: 403
https://www.elcorteingles.es/bricor/armarios/armarios-multiusos/
Status code error on url https://www.elcorteingles.es/bricor/armarios/armarios-multiusos/. Status code: 403
https://www.elcorteingles.es/bricor/armarios/mobiliario-dormitorio/
Status code error on url https://www.elcorteingles.es/bricor/armarios/mobiliario-dormitorio/. Status code: 403
https://www.elcorteingles.es/bricor/armarios/caj

In [20]:
len(products)

110496

In [27]:
products[110495]

{'id': '001097631102874',
 'brand': 'IMC Toys',
 'store_id': '38',
 'position': 23,
 'badges': ['express_delivery'],
 'media': {'count': 1},
 'name': 'Pilla ratón IMC Toys',
 'variant': '001097631102874   ',
 'category': ['Juguetes', 'Juegos de mesa', 'Habilidad'],
 'alternative_id': 'A14042923',
 'eci_provider': '00000000',
 'gtin': '8421134007413',
 'price': {'final': 44, 'currency': 'EUR'},
 'status': 'add',
 'quantity': 1,
 'image': 'http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/201610/28/00197631102874____4__640x640.jpg'}

Lo guardamos en un Dataframe para poder usarlo en batch

In [28]:
dfProducts = pd.DataFrame.from_records(products)
dfProducts.head(3)

,alternative_id,availability_date,badges,brand,category,discount,eci_provider,gtin,id,image,media,name,position,price,quantity,status,store_id,variant
0,A28577025,NaN,[express_delivery],Roberto Verino,"[Moda, Mujer, Abrigos]",NaN,00000000,8433815612144,001024836201442,http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03...,{'count': 1},Parka corta de mujer Roberto Verino con estamp...,0.0,"{'final': 220, 'currency': 'EUR'}",1,show_pdp,60,001024836201442038
1,A28577035,NaN,[express_delivery],Roberto Verino,"[Moda, Mujer, Abrigos]",NaN,00000000,8433815612380,001024837200104,http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03...,{'count': 1},Parka ligera de mujer Roberto Verino en beige,1.0,"{'final': 220, 'currency': 'EUR'}",1,show_pdp,60,001024837200104038
2,A28577038,NaN,[express_delivery],Roberto Verino,"[Moda, Mujer, Abrigos]",NaN,00000000,8433815612465,001024837200112,http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03...,{'count': 1},Parka larga de mujer Roberto Verino con capucha,2.0,"{'final': 220, 'currency': 'EUR'}",1,show_pdp,60,001024837200112038


In [29]:
dfProducts.shape

(110496, 18)

### Quitar duplicados

In [30]:
dfProducts = dfProducts.drop_duplicates(subset='id')
dfProducts.shape

(59607, 18)

In [31]:
dfProducts.to_csv('Products.csv', index=False)

## Save images

In [24]:
for n, product in enumerate (products):
    pic_url = product['image']
    print (n)
    
    name = ''
    for i in product['category']:
        name += i+'_'
    name = name + product['id'] + '.jpg'
    
    with open('Images/'+name, 'wb') as img_file:
        response = requests.get(pic_url, stream=True)

        if response.ok:

            for batch in response.iter_content(1024):
                if not batch:
                    break

                img_file.write(batch)
        else:
            print(f'Status code error with {response.status_code} code.')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

Top 20 Marcas

In [78]:
dfProducts['brand'].value_counts()[:20]

El Corte Inglés         5062
Tommy Hilfiger          1421
Freestyle               1087
Polo Ralph Lauren       1070
Emidio Tucci            1043
Nike                    1026
Fox Home                 918
Gloria Ortiz             830
Dustin                   824
MICHAEL Michael Kors     804
Easy Wear                777
Dulces                   757
GAP                      725
Tous                     713
Sony                     697
Brotes                   692
Tutto Piccolo            674
Tuc tuc                  674
Énfasis                  664
Philips                  645
Name: brand, dtype: int64